In [12]:
%matplotlib inline
import matplotlib.pyplot as plt

In [13]:
import math
import pandas as pd
import numpy as np
import csv
import datetime

def calculate_initial_compass_bearing(pointA, pointB):
    """
    From https://gist.github.com/jeromer/2005586
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing


def createbear_outputnew(filename,newfile):
    
    GPS = pd.read_csv(filename,skiprows=19,error_bad_lines=False)
    GPS = GPS[:-6] # skipping the last six row because that's not useful
    timestamp = GPS['timestamp(s)']
    lat = GPS['position_lat(semicircles)']
    lon = GPS['position_long(semicircles)']
    spd = GPS['speed(m/s)']
    dist = GPS['distance(m)']
    
    # correcting the timestamp
    gpstime = []
    for i in range(0,len(timestamp)):
        ddt = datetime.datetime.strptime(timestamp[i],"%Y-%m-%d %H:%M:%S")
        adj = ddt + datetime.timedelta(hours=-7)
        gpstime = np.append(str(adj),gpstime)
    gpstime = gpstime[::-1]
    
    # converting from Garmin semicircles to degree coordinates
    lat_deg = []
    for item in lat:
        new = float(item)*(180/float(np.power(2,31)))
        lat_deg = np.append(new,lat_deg)
    lat_deg=lat_deg[::-1]

    lon_deg = []
    for item in lon:
        new2 = float(item)*(180/float(np.power(2,31)))
        lon_deg = np.append(new2,lon_deg)
    lon_deg=lon_deg[::-1]
    
    bear = []
    for i in range(len(lat_deg)-1):
        bear = np.append(calculate_initial_compass_bearing((lat_deg[i],lon_deg[i]),(lat_deg[i+1],lon_deg[i+1])),bear)
    bear = bear[::-1]
    bear = np.append(0,bear) # add on the first bear as zero
    
     # write to new .csv files
    rows = zip(gpstime,lat_deg,lon_deg,spd,bear,dist)
    with open(newfile, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(['GPStime','lat(DD)','lon(DD)','speed(m/s)','bearing(deg)','distance(m)'])
        for i in rows:
            writer.writerow(i)

In [14]:
def createbear_outputnew_newhour(filename,newfile):
    
    '''
    On May 24, found that four days have incorrect times by one hour. Made this for them.
    '''
    
    GPS = pd.read_csv(filename,skiprows=19,error_bad_lines=False)
    GPS = GPS[:-6] # skipping the last six row because that's not useful
    timestamp = GPS['timestamp(s)']
    lat = GPS['position_lat(semicircles)']
    lon = GPS['position_long(semicircles)']
    spd = GPS['speed(m/s)']
    dist = GPS['distance(m)']
    
    # correcting the timestamp
    gpstime = []
    for i in range(0,len(timestamp)):
        ddt = datetime.datetime.strptime(timestamp[i],"%Y-%m-%d %H:%M:%S")
        adj = ddt + datetime.timedelta(hours=-6)
        gpstime = np.append(str(adj),gpstime)
    gpstime = gpstime[::-1]
    
    # converting from Garmin semicircles to degree coordinates
    lat_deg = []
    for item in lat:
        new = float(item)*(180/float(np.power(2,31)))
        lat_deg = np.append(new,lat_deg)
    lat_deg=lat_deg[::-1]

    lon_deg = []
    for item in lon:
        new2 = float(item)*(180/float(np.power(2,31)))
        lon_deg = np.append(new2,lon_deg)
    lon_deg=lon_deg[::-1]
    
    bear = []
    for i in range(len(lat_deg)-1):
        bear = np.append(calculate_initial_compass_bearing((lat_deg[i],lon_deg[i]),(lat_deg[i+1],lon_deg[i+1])),bear)
    bear = bear[::-1]
    bear = np.append(0,bear) # add on the first bear as zero
    
     # write to new .csv files
    rows = zip(gpstime,lat_deg,lon_deg,spd,bear,dist)
    with open(newfile, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(['GPStime','lat(DD)','lon(DD)','speed(m/s)','bearing(deg)','distance(m)'])
        for i in rows:
            writer.writerow(i)

In [15]:
createbear_outputnew('From_conversion_website/2015-10-02-15-29-45.csv','BEAR/Oct2.csv')

Skipping line 180: expected 9 fields, saw 17
Skipping line 181: expected 9 fields, saw 17
Skipping line 182: expected 9 fields, saw 17
Skipping line 183: expected 9 fields, saw 111
Skipping line 184: expected 9 fields, saw 37
Skipping line 185: expected 9 fields, saw 37
Skipping line 187: expected 9 fields, saw 123
Skipping line 188: expected 9 fields, saw 41
Skipping line 189: expected 9 fields, saw 41
Skipping line 190: expected 9 fields, saw 24



In [16]:
createbear_outputnew('From_conversion_website/2015-10-14-15-09-08.csv','BEAR/Oct14.csv')

Skipping line 262: expected 9 fields, saw 12
Skipping line 263: expected 9 fields, saw 12
Skipping line 565: expected 9 fields, saw 17
Skipping line 566: expected 9 fields, saw 17
Skipping line 567: expected 9 fields, saw 17
Skipping line 568: expected 9 fields, saw 111
Skipping line 569: expected 9 fields, saw 37
Skipping line 570: expected 9 fields, saw 37
Skipping line 572: expected 9 fields, saw 123
Skipping line 573: expected 9 fields, saw 41
Skipping line 574: expected 9 fields, saw 41
Skipping line 575: expected 9 fields, saw 24



In [25]:
createbear_outputnew('From_conversion_website/2015-11-02-15-34-48.csv','BEAR/Nov2.csv')

Skipping line 389: expected 9 fields, saw 12
Skipping line 390: expected 9 fields, saw 12
Skipping line 490: expected 9 fields, saw 17
Skipping line 491: expected 9 fields, saw 17
Skipping line 492: expected 9 fields, saw 17
Skipping line 493: expected 9 fields, saw 111
Skipping line 494: expected 9 fields, saw 37
Skipping line 495: expected 9 fields, saw 37
Skipping line 497: expected 9 fields, saw 123
Skipping line 498: expected 9 fields, saw 41
Skipping line 499: expected 9 fields, saw 41
Skipping line 500: expected 9 fields, saw 24



In [26]:
createbear_outputnew('From_conversion_website/2015-11-03-16-41-16.csv','BEAR/Nov3.csv')

Skipping line 89: expected 9 fields, saw 12
Skipping line 90: expected 9 fields, saw 12
Skipping line 384: expected 9 fields, saw 12
Skipping line 385: expected 9 fields, saw 12
Skipping line 525: expected 9 fields, saw 17
Skipping line 526: expected 9 fields, saw 17
Skipping line 527: expected 9 fields, saw 17
Skipping line 528: expected 9 fields, saw 111
Skipping line 529: expected 9 fields, saw 37
Skipping line 530: expected 9 fields, saw 37
Skipping line 532: expected 9 fields, saw 123
Skipping line 533: expected 9 fields, saw 41
Skipping line 534: expected 9 fields, saw 41
Skipping line 535: expected 9 fields, saw 24



In [27]:
createbear_outputnew('From_conversion_website/2015-11-04-15-46-57.csv','BEAR/Nov4.csv')

Skipping line 206: expected 9 fields, saw 12
Skipping line 207: expected 9 fields, saw 12
Skipping line 618: expected 9 fields, saw 12
Skipping line 619: expected 9 fields, saw 12
Skipping line 622: expected 9 fields, saw 17
Skipping line 623: expected 9 fields, saw 17
Skipping line 624: expected 9 fields, saw 17
Skipping line 625: expected 9 fields, saw 111
Skipping line 626: expected 9 fields, saw 37
Skipping line 627: expected 9 fields, saw 37
Skipping line 629: expected 9 fields, saw 123
Skipping line 630: expected 9 fields, saw 41
Skipping line 631: expected 9 fields, saw 41
Skipping line 632: expected 9 fields, saw 24



In [28]:
createbear_outputnew('From_conversion_website/2015-11-10-15-49-11.csv','BEAR/Nov10.csv')

Skipping line 57: expected 9 fields, saw 12
Skipping line 58: expected 9 fields, saw 12
Skipping line 440: expected 9 fields, saw 12
Skipping line 441: expected 9 fields, saw 12
Skipping line 537: expected 9 fields, saw 17
Skipping line 538: expected 9 fields, saw 17
Skipping line 539: expected 9 fields, saw 17
Skipping line 540: expected 9 fields, saw 111
Skipping line 541: expected 9 fields, saw 37
Skipping line 542: expected 9 fields, saw 37
Skipping line 544: expected 9 fields, saw 123
Skipping line 545: expected 9 fields, saw 41
Skipping line 546: expected 9 fields, saw 41
Skipping line 547: expected 9 fields, saw 24



In [29]:
createbear_outputnew('From_conversion_website/2016-02-19-16-05-50.csv','BEAR/Feb19.csv')

Skipping line 170: expected 9 fields, saw 12
Skipping line 171: expected 9 fields, saw 12
Skipping line 521: expected 9 fields, saw 17
Skipping line 522: expected 9 fields, saw 17
Skipping line 523: expected 9 fields, saw 17
Skipping line 524: expected 9 fields, saw 111
Skipping line 525: expected 9 fields, saw 37
Skipping line 526: expected 9 fields, saw 37
Skipping line 528: expected 9 fields, saw 123
Skipping line 529: expected 9 fields, saw 41
Skipping line 530: expected 9 fields, saw 41
Skipping line 531: expected 9 fields, saw 24



In [30]:
createbear_outputnew('From_conversion_website/2016-02-20-15-39-14.csv','BEAR/Feb20.csv')

Skipping line 204: expected 9 fields, saw 12
Skipping line 205: expected 9 fields, saw 12
Skipping line 602: expected 9 fields, saw 17
Skipping line 603: expected 9 fields, saw 17
Skipping line 604: expected 9 fields, saw 17
Skipping line 605: expected 9 fields, saw 111
Skipping line 606: expected 9 fields, saw 37
Skipping line 607: expected 9 fields, saw 37
Skipping line 609: expected 9 fields, saw 123
Skipping line 610: expected 9 fields, saw 41
Skipping line 611: expected 9 fields, saw 41
Skipping line 612: expected 9 fields, saw 24



In [21]:
createbear_outputnew('From_conversion_website/2016-03-02-16-00-37.csv','BEAR/Mar2.csv')

In [32]:
createbear_outputnew('From_conversion_website/2016-03-04-16-04-51.csv','BEAR/Mar4.csv')

Skipping line 259: expected 9 fields, saw 12
Skipping line 260: expected 9 fields, saw 12
Skipping line 537: expected 9 fields, saw 17
Skipping line 538: expected 9 fields, saw 17
Skipping line 539: expected 9 fields, saw 17
Skipping line 540: expected 9 fields, saw 111
Skipping line 541: expected 9 fields, saw 37
Skipping line 542: expected 9 fields, saw 37
Skipping line 544: expected 9 fields, saw 123
Skipping line 545: expected 9 fields, saw 41
Skipping line 546: expected 9 fields, saw 41
Skipping line 547: expected 9 fields, saw 24



In [33]:
createbear_outputnew('From_conversion_website/2016-03-07-16-20-48.csv','BEAR/Mar7.csv')

Skipping line 415: expected 9 fields, saw 12
Skipping line 416: expected 9 fields, saw 12
Skipping line 582: expected 9 fields, saw 17
Skipping line 583: expected 9 fields, saw 17
Skipping line 584: expected 9 fields, saw 17
Skipping line 585: expected 9 fields, saw 111
Skipping line 586: expected 9 fields, saw 37
Skipping line 587: expected 9 fields, saw 37
Skipping line 589: expected 9 fields, saw 123
Skipping line 590: expected 9 fields, saw 41
Skipping line 591: expected 9 fields, saw 41
Skipping line 592: expected 9 fields, saw 24



In [8]:
createbear_outputnew_newhour('From_conversion_website/2016-03-14-16-10-38.csv','BEAR/Mar14.csv')

Skipping line 106: expected 9 fields, saw 12
Skipping line 107: expected 9 fields, saw 12
Skipping line 151: expected 9 fields, saw 12
Skipping line 152: expected 9 fields, saw 12
Skipping line 686: expected 9 fields, saw 17
Skipping line 687: expected 9 fields, saw 17
Skipping line 688: expected 9 fields, saw 17
Skipping line 689: expected 9 fields, saw 111
Skipping line 690: expected 9 fields, saw 37
Skipping line 691: expected 9 fields, saw 37
Skipping line 693: expected 9 fields, saw 123
Skipping line 694: expected 9 fields, saw 41
Skipping line 695: expected 9 fields, saw 41
Skipping line 696: expected 9 fields, saw 24



In [9]:
createbear_outputnew_newhour('From_conversion_website/2016-03-21-16-23-46.csv','BEAR/Mar21.csv')

Skipping line 127: expected 9 fields, saw 12
Skipping line 128: expected 9 fields, saw 12
Skipping line 353: expected 9 fields, saw 12
Skipping line 354: expected 9 fields, saw 12
Skipping line 630: expected 9 fields, saw 17
Skipping line 631: expected 9 fields, saw 17
Skipping line 632: expected 9 fields, saw 17
Skipping line 633: expected 9 fields, saw 111
Skipping line 634: expected 9 fields, saw 37
Skipping line 635: expected 9 fields, saw 37
Skipping line 637: expected 9 fields, saw 123
Skipping line 638: expected 9 fields, saw 41
Skipping line 639: expected 9 fields, saw 41
Skipping line 640: expected 9 fields, saw 24



In [10]:
createbear_outputnew_newhour('From_conversion_website/2016-03-22-15-54-05.csv','BEAR/Mar22.csv')

Skipping line 240: expected 9 fields, saw 12
Skipping line 241: expected 9 fields, saw 12
Skipping line 763: expected 9 fields, saw 17
Skipping line 764: expected 9 fields, saw 17
Skipping line 765: expected 9 fields, saw 17
Skipping line 766: expected 9 fields, saw 111
Skipping line 767: expected 9 fields, saw 37
Skipping line 768: expected 9 fields, saw 37
Skipping line 770: expected 9 fields, saw 123
Skipping line 771: expected 9 fields, saw 41
Skipping line 772: expected 9 fields, saw 41
Skipping line 773: expected 9 fields, saw 24



In [11]:
createbear_outputnew_newhour('From_conversion_website/2016-03-25-15-52-54.csv','BEAR/Mar25.csv')

Skipping line 112: expected 9 fields, saw 12
Skipping line 113: expected 9 fields, saw 12
Skipping line 190: expected 9 fields, saw 12
Skipping line 191: expected 9 fields, saw 12
Skipping line 567: expected 9 fields, saw 12
Skipping line 568: expected 9 fields, saw 12
Skipping line 693: expected 9 fields, saw 17
Skipping line 694: expected 9 fields, saw 17
Skipping line 695: expected 9 fields, saw 17
Skipping line 696: expected 9 fields, saw 111
Skipping line 697: expected 9 fields, saw 37
Skipping line 698: expected 9 fields, saw 37
Skipping line 700: expected 9 fields, saw 123
Skipping line 701: expected 9 fields, saw 41
Skipping line 702: expected 9 fields, saw 41
Skipping line 703: expected 9 fields, saw 24



In [52]:
createbear_outputnew('From_conversion_website/2016-03-28-16-15-05.csv','BEAR/Mar28.csv')

Skipping line 609: expected 9 fields, saw 12
Skipping line 610: expected 9 fields, saw 12
Skipping line 682: expected 9 fields, saw 17
Skipping line 683: expected 9 fields, saw 17
Skipping line 684: expected 9 fields, saw 17
Skipping line 685: expected 9 fields, saw 111
Skipping line 686: expected 9 fields, saw 37
Skipping line 687: expected 9 fields, saw 37
Skipping line 689: expected 9 fields, saw 123
Skipping line 690: expected 9 fields, saw 41
Skipping line 691: expected 9 fields, saw 41
Skipping line 692: expected 9 fields, saw 24



In [64]:
createbear_outputnew('From_conversion_website/2016-03-29-16-00-50.csv','BEAR/Mar29.csv')
# No errors for unknown reason. I made the file from the previous day I think. Removed nans manually

In [65]:
createbear_outputnew('From_conversion_website/2016-03-30-16-20-52.csv','BEAR/Mar30.csv')

Skipping line 635: expected 9 fields, saw 12
Skipping line 636: expected 9 fields, saw 12
Skipping line 883: expected 9 fields, saw 17
Skipping line 884: expected 9 fields, saw 17
Skipping line 885: expected 9 fields, saw 17
Skipping line 886: expected 9 fields, saw 111
Skipping line 887: expected 9 fields, saw 37
Skipping line 888: expected 9 fields, saw 37
Skipping line 890: expected 9 fields, saw 123
Skipping line 891: expected 9 fields, saw 41
Skipping line 892: expected 9 fields, saw 41
Skipping line 893: expected 9 fields, saw 24



In [66]:
createbear_outputnew('From_conversion_website/2016-04-04-16-08-24.csv','BEAR/Apr4.csv')

Skipping line 539: expected 9 fields, saw 12
Skipping line 540: expected 9 fields, saw 12
Skipping line 604: expected 9 fields, saw 17
Skipping line 605: expected 9 fields, saw 17
Skipping line 606: expected 9 fields, saw 17
Skipping line 607: expected 9 fields, saw 111
Skipping line 608: expected 9 fields, saw 37
Skipping line 609: expected 9 fields, saw 37
Skipping line 611: expected 9 fields, saw 123
Skipping line 612: expected 9 fields, saw 41
Skipping line 613: expected 9 fields, saw 41
Skipping line 614: expected 9 fields, saw 24



In [67]:
createbear_outputnew('From_conversion_website/2016-04-05-16-24-36.csv','BEAR/Apr5.csv')

Skipping line 376: expected 9 fields, saw 12
Skipping line 377: expected 9 fields, saw 12
Skipping line 641: expected 9 fields, saw 12
Skipping line 642: expected 9 fields, saw 12
Skipping line 679: expected 9 fields, saw 17
Skipping line 680: expected 9 fields, saw 17
Skipping line 681: expected 9 fields, saw 17
Skipping line 682: expected 9 fields, saw 111
Skipping line 683: expected 9 fields, saw 37
Skipping line 684: expected 9 fields, saw 37
Skipping line 686: expected 9 fields, saw 123
Skipping line 687: expected 9 fields, saw 41
Skipping line 688: expected 9 fields, saw 41
Skipping line 689: expected 9 fields, saw 24



In [68]:
createbear_outputnew('From_conversion_website/2016-04-06-16-14-30.csv','BEAR/Apr6.csv')

Skipping line 555: expected 9 fields, saw 17
Skipping line 556: expected 9 fields, saw 17
Skipping line 557: expected 9 fields, saw 17
Skipping line 558: expected 9 fields, saw 111
Skipping line 559: expected 9 fields, saw 37
Skipping line 560: expected 9 fields, saw 37
Skipping line 562: expected 9 fields, saw 123
Skipping line 563: expected 9 fields, saw 41
Skipping line 564: expected 9 fields, saw 41
Skipping line 565: expected 9 fields, saw 24



In [69]:
createbear_outputnew('From_conversion_website/2016-04-07-16-31-39.csv','BEAR/Apr7.csv')

Skipping line 803: expected 9 fields, saw 17
Skipping line 804: expected 9 fields, saw 17
Skipping line 805: expected 9 fields, saw 17
Skipping line 806: expected 9 fields, saw 111
Skipping line 807: expected 9 fields, saw 37
Skipping line 808: expected 9 fields, saw 37
Skipping line 810: expected 9 fields, saw 123
Skipping line 811: expected 9 fields, saw 41
Skipping line 812: expected 9 fields, saw 41
Skipping line 813: expected 9 fields, saw 24



In [70]:
createbear_outputnew('From_conversion_website/2016-04-13-16-10-26.csv','BEAR/Apr13.csv')

Skipping line 487: expected 9 fields, saw 12
Skipping line 488: expected 9 fields, saw 12
Skipping line 557: expected 9 fields, saw 17
Skipping line 558: expected 9 fields, saw 17
Skipping line 559: expected 9 fields, saw 17
Skipping line 560: expected 9 fields, saw 111
Skipping line 561: expected 9 fields, saw 37
Skipping line 562: expected 9 fields, saw 37
Skipping line 564: expected 9 fields, saw 123
Skipping line 565: expected 9 fields, saw 41
Skipping line 566: expected 9 fields, saw 41
Skipping line 567: expected 9 fields, saw 24



In [71]:
createbear_outputnew('From_conversion_website/2016-04-14-16-14-11.csv','BEAR/Apr14.csv')

Skipping line 181: expected 9 fields, saw 12
Skipping line 182: expected 9 fields, saw 12
Skipping line 587: expected 9 fields, saw 17
Skipping line 588: expected 9 fields, saw 17
Skipping line 589: expected 9 fields, saw 17
Skipping line 590: expected 9 fields, saw 111
Skipping line 591: expected 9 fields, saw 37
Skipping line 592: expected 9 fields, saw 37
Skipping line 594: expected 9 fields, saw 123
Skipping line 595: expected 9 fields, saw 41
Skipping line 596: expected 9 fields, saw 41
Skipping line 597: expected 9 fields, saw 24



In [72]:
createbear_outputnew('From_conversion_website/2016-04-15-15-56-50.csv','BEAR/Apr15.csv')

Skipping line 609: expected 9 fields, saw 17
Skipping line 610: expected 9 fields, saw 17
Skipping line 611: expected 9 fields, saw 17
Skipping line 612: expected 9 fields, saw 111
Skipping line 613: expected 9 fields, saw 37
Skipping line 614: expected 9 fields, saw 37
Skipping line 616: expected 9 fields, saw 123
Skipping line 617: expected 9 fields, saw 41
Skipping line 618: expected 9 fields, saw 41
Skipping line 619: expected 9 fields, saw 24



In [80]:
createbear_outputnew('From_conversion_website/2016-04-27-16-15-54.csv','BEAR/Apr27.csv')
# JUST LIKE MAR29, MUST REMOVE nans IF RUN THIS AGAIN. Manually edited file

In [83]:
createbear_outputnew('From_conversion_website/2016-04-28-16-09-34.csv','BEAR/Apr28.csv')
# JUST LIKE MAR29, MUST REMOVE nans IF RUN THIS AGAIN. Manually edited file

In [84]:
createbear_outputnew('From_conversion_website/2016-04-29-15-59-06.csv','BEAR/Apr29.csv')

Skipping line 222: expected 9 fields, saw 12
Skipping line 223: expected 9 fields, saw 12
Skipping line 962: expected 9 fields, saw 17
Skipping line 963: expected 9 fields, saw 17
Skipping line 964: expected 9 fields, saw 17
Skipping line 965: expected 9 fields, saw 111
Skipping line 966: expected 9 fields, saw 37
Skipping line 967: expected 9 fields, saw 37
Skipping line 969: expected 9 fields, saw 123
Skipping line 970: expected 9 fields, saw 41
Skipping line 971: expected 9 fields, saw 41
Skipping line 972: expected 9 fields, saw 24



In [85]:
createbear_outputnew('From_conversion_website/2016-05-05-15-57-04.csv','BEAR/May5.csv')

Skipping line 350: expected 9 fields, saw 12
Skipping line 351: expected 9 fields, saw 12
Skipping line 612: expected 9 fields, saw 17
Skipping line 613: expected 9 fields, saw 17
Skipping line 614: expected 9 fields, saw 17
Skipping line 615: expected 9 fields, saw 111
Skipping line 616: expected 9 fields, saw 37
Skipping line 617: expected 9 fields, saw 37
Skipping line 619: expected 9 fields, saw 123
Skipping line 620: expected 9 fields, saw 41
Skipping line 621: expected 9 fields, saw 41
Skipping line 622: expected 9 fields, saw 24



In [86]:
createbear_outputnew('From_conversion_website/2016-05-06-15-56-36.csv','BEAR/May6.csv')

Skipping line 387: expected 9 fields, saw 12
Skipping line 388: expected 9 fields, saw 12
Skipping line 997: expected 9 fields, saw 17
Skipping line 998: expected 9 fields, saw 17
Skipping line 999: expected 9 fields, saw 17
Skipping line 1000: expected 9 fields, saw 111
Skipping line 1001: expected 9 fields, saw 37
Skipping line 1002: expected 9 fields, saw 37
Skipping line 1004: expected 9 fields, saw 123
Skipping line 1005: expected 9 fields, saw 41
Skipping line 1006: expected 9 fields, saw 41
Skipping line 1007: expected 9 fields, saw 24

